# A Population Model

This section introduces a simplified **population model** designed to illustrate how *ModelFlow* handles **cohort dynamics**, **age transitions**, and **causal dependencies**.

The model tracks population changes across age groups and sexes, incorporating key demographic processes such as:

- **Births** — introducing new individuals into the youngest cohort  
- **Deaths** — removing individuals from each age group based on age-specific mortality rates  
- **Migration** — adjusting population levels through net inflows or outflows  
- **Aging** — transferring individuals from one age group to the next over time  

Through automatic list expansion and dependency tracing, *ModelFlow* enables clear and compact equation definitions while maintaining full transparency of the underlying causal structure.

In [41]:
from pprint import pprint

from modelconstruct import Mexplode
from modelpattern import list_extract

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
pprint(dict(list_extract('''list ages = ages : age_0 * age_3    $''')))

## Derived sublists
To support **modeling of cohorts** and **debt/asset developments**, *ModelFlow* automatically creates a set of derived sublists based on an initial list definition.

For example, the statement:

> list ages = ages : age_0 * age_3

defines a base list and generates several auxiliary lists that make it easier to model age transitions and boundary conditions.

The following sublists become available:

| **Key**          | **Description / Interpretation**              | **Values**                 |
|------------------|-----------------------------------------------|----------------------------|
| `AGES`           | The base list of age variables                | `['AGE_0', 'AGE_1', 'AGE_2']` |
| `AGES_AFTER`     | Elements shifted one step forward (next age)  | `['AGE_1', 'AGE_2', '0']`     |
| `AGES_BEFORE`    | Elements shifted one step backward (previous age) | `['0', 'AGE_0', 'AGE_1']` |
| `AGES_END`       | Flag: 1 if last element, else 0               | `['0', '0', '1']`          |
| `AGES_MIDDLE`    | Flag: 1 if middle element(s), else 0          | `['0', '1', '0']`          |
| `AGES_NOEND`     | Flag: 1 if not the last element, else 0       | `['1', '1', '0']`          |
| `AGES_NOSTART`   | Flag: 1 if not the first element, else 0      | `['0', '1', '1']`          |
| `AGES_START`     | Flag: 1 if first element, else 0              | `['1', '0', '0']`          |


### Example usage

These derived lists can be used both for **shifting variables** across age or maturity groups and for **conditional equation generation**.

For instance, the statement below describes that the **population at the start of a year (`pop_primo`)** equals the **population in the same age group at the end of the previous year (`pop`)**, but **only for age groups that are not the first**:

```python
doable [ages ages_nostart]  pop_primo__{ages} = pop__{ages_before}
```

Here:

 - ages iterates over all defined age groups (AGE_0, AGE_1, AGE_2).

 - ages_nostart ensures that the equation is not applied to the first group (AGE_0).

 - ages_before provides the corresponding previous age ( AGE_1 → AGE_0, etc.).

This structure allows concise specification of interlinked cohort or maturity dynamics while automatically handling boundary conditions such as the first or last group.

In [ ]:
Mexplode('''
list ages = ages : age_0 * age_5
doable [ages ages_nostart]  pop_primo__{ages} = pop__{ages_before}(-1)''').show

## Specification of a model describing population dynamics

Below, a small population model is defined.  
To make the specification more concise and readable, two string replacements are performed through the call to `Mexplode`.

The replacements are:

| **Placeholder** | **Replaced with**                         | **Description** |
|:------------------|:-------------------------------------------|:-----------------|
| `__ab`           | `__{sexes}__{ages_before}`                | Refers to the same sex but the previous age group |
| `__a`            | `__{sexes}__{ages}`                       | Refers to the same sex and the current age group |

This approach reduces visual clutter in the equation definitions and helps express structured relationships more clearly across **sex** and **age** dimensions.


In [45]:
eq = '''
list ages = ages : age_0 * age_11 
list sexes = sexes   : female male /
             fertile :     1     0
£ primo population is (ultimo)population in age group before
doable [ages ages_nostart]  pop_primo__a = pop__ab(-1)

£ for first age group the population depends on all births 
doable [ages ages_start]   pop_before_death__a   = birth__total__{sexes} + migration__a
doable [ages ages_nostart] pop_before_death__a   = pop_primo__a + migration__a 

£ --- Step 3: Deaths ---
doable  [ages ages_end ]  death_rate__a =1.0 
doable  <sum=total>          death__a = pop_before_death__a  * death_rate__a

£ --- Step 4: End-of-period population ---
doable  <sum=total>          pop__a  = pop_before_death__a  - death__a 

£ --- Step 5: Births  (all except AGE_0 ) fertility is assumed 0 for low and high ages  ---

doable <sum=total> [ages ages_nostart,  sexes    fertile] birth__a = pop_primo__a * fertility__a

birth_total = sum(ages ages_nostart=1,sum(sexes fertile=1,birth__a))        

£ --- Some are born female and some are born male :

birth__total__female = FRAC_BIRTH__female * birth_total
birth__total__male = (1 - FRAC_BIRTH__female) * birth_total


£ --- Step 6: Totals and consistency check ---
migration__total = sum(ages,sum(sexes,migration__a))
pop__total_check  = (pop__total(-1) + birth__total + migration__total - death__total) - pop__total


'''
print('This Input:')
(mm := Mexplode(eq,
replacements=[('__ab','__{sexes}__{ages_before}'),('__a','__{sexes}__{ages}')]
               
               )).showclean_frml_statements  
print('\nCreates this Output')
mm.show

This Input:
Clean_frml_statements: 
LIST AGES =
    AGES : AGE_0 * AGE_11  $
LIST SEXES =
    SEXES   : FEMALE MALE /
    FERTILE : 1      0  $
£ PRIMO POPULATION IS (ULTIMO)POPULATION IN AGE GROUP BEFORE
DOABLE [AGES AGES_NOSTART]  POP_PRIMO__{SEXES}__{AGES} = POP__{SEXES}__{AGES_BEFORE}(-1)$

£ FOR FIRST AGE GROUP THE POPULATION DEPENDS ON ALL BIRTHS 
DOABLE [AGES AGES_START]   POP_BEFORE_DEATH__{SEXES}__{AGES}   = BIRTH__TOTAL__{SEXES} + MIGRATION__{SEXES}__{AGES}$
DOABLE [AGES AGES_NOSTART] POP_BEFORE_DEATH__{SEXES}__{AGES}   = POP_PRIMO__{SEXES}__{AGES} + MIGRATION__{SEXES}__{AGES} $

£ --- STEP 3: DEATHS ---
DOABLE  [AGES AGES_END ]  DEATH_RATE__{SEXES}__{AGES} =1.0 $
DOABLE  <SUM=TOTAL>          DEATH__{SEXES}__{AGES} = POP_BEFORE_DEATH__{SEXES}__{AGES}  * DEATH_RATE__{SEXES}__{AGES}$

£ --- STEP 4: END-OF-PERIOD POPULATION ---
DOABLE  <SUM=TOTAL>          POP__{SEXES}__{AGES}  = POP_BEFORE_DEATH__{SEXES}__{AGES}  - DEATH__{SEXES}__{AGES} $

£ --- STEP 5: BIRTHS  (ALL EXCEPT AGE

## Tracing the causality structure

In the example below, the **causal chain** leading to the variable `POP__MALE__AGE_2` is traced upward through **8 dependency levels**.

>mm.mmodel.pop__male__age_2.tracepre(up=8, size=(10,10))   

This produces a causal dependency graph (as shown below), illustrating how upstream variables contribute to the population of 2-year-old males:

 - Blue boxes represent endogenous population and mortality variables.

 - Yellow boxes indicate exogenous , such as migration and death rates.

 - The red box highlights the target variable (POP__MALE__AGE_2).

The arrows show the flow of causality, moving from fundamental drivers like births, migration, and mortality rates toward the resulting population cohort.
This visualization helps understand how demographic processes propagate through the model and affect specific age groups.

In [44]:
mm.mmodel.pop__male__age_2.tracepre(up=8,size=(10,10))